In [106]:
import numpy as np
import random
import math
import copy

# Task 1: Generate 40 trucks with capacity between 500 to 1000Kg
trucks_capacity = np.random.randint(500, 1001, size=40)

# Task 2: Generate 120 packages with weight (between 200-1000) and distance (between 20 to 100)
packages = np.random.randint(200, 951, size=(120, 2))
packages[:, 1] = np.random.randint(20, 101, size=120)

# Task 3: Initial assignment
def initial_assignment(trucks_capacity, packages):
    assignments = np.full((40, 3, 3), -1, dtype=int)

    package_assigned = np.zeros(len(packages), dtype=bool)
    for t_idx, capacity in enumerate(trucks_capacity):
        for trip in range(3):  # A truck can make 3 trips
            remaining_capacity = capacity
            for i in range(3):  # A truck can carry up to 3 packages per trip
                # Find packages that can be carried by the current truck and are not yet assigned
                available_packages = np.argwhere((packages[:, 0] <= remaining_capacity) & (~package_assigned)).flatten()
                if len(available_packages) > 0:
                    p_idx = np.random.choice(available_packages)
                    assignments[t_idx][trip][i] = p_idx
                    package_assigned[p_idx] = True
                    remaining_capacity -= packages[p_idx][0]
                else:
                    break

    return assignments, package_assigned

# Task 4: Calculate the cost
def calculate_cost(assignments, initial_capacity, packages):
    cost = 0
    delivered_packages = np.zeros(len(packages), dtype=bool)
    for t_idx, trips in enumerate(assignments):
        for trip in trips:
            for p_idx in trip:
                if p_idx != -1:
                    p = packages[p_idx]
                    cost += (initial_capacity[t_idx] - p[0]) / p[1]
                    delivered_packages[p_idx] = True

    # Add penalty for undelivered packages
    penalty = 100 * np.sum(np.logical_not(delivered_packages))
    cost += penalty

    return cost, delivered_packages

# Task 5 and 6: Use SGA optimization
def optimization(assignments, trucks_capacity, packages, initial_cost, max_iter=7500, tol=0.5):
    initial_capacity = trucks_capacity.copy()
    current_cost = initial_cost
    current_assignments = copy.deepcopy(assignments)

    for iteration in range(max_iter):
        # Swap two random packages
        t1, t2 = random.sample(range(40), 2)
        trip1, trip2 = random.sample(range(3), 2)
        p1, p2 = random.sample(range(3), 2)
        new_assignments = copy.deepcopy(current_assignments)
        new_assignments[t1][trip1][p1], new_assignments[t2][trip2][p2] = new_assignments[t2][trip2][p2], new_assignments[t1][trip1][p1]

        # Check if the swap exceeds the truck capacity
        new_assigned_weights_t1 = [packages[p_idx][0] if p_idx != -1 else 0 for p_idx in new_assignments[t1][trip1]]
        new_assigned_weights_t2 = [packages[p_idx][0] if p_idx != -1 else 0 for p_idx in new_assignments[t2][trip2]]
        if sum(new_assigned_weights_t1) > initial_capacity[t1] or sum(new_assigned_weights_t2) > initial_capacity[t2]:
            continue

        # Calculate new cost
        new_cost, delivered_packages = calculate_cost(new_assignments, initial_capacity, packages)

        # Check if the cost is better
        if new_cost < current_cost:
            print(f"Iteration {iteration}: Switched package {p1+1} of truck {t1+1} trip {trip1+1} with package {p2+1} of truck {t2+1} trip {trip2+1}, old cost: {current_cost}, new cost: {new_cost}")
            # Check for convergence
            if abs(new_cost - current_cost) < tol and np.all(delivered_packages):
                break
            current_assignments, current_cost = new_assignments, new_cost

        # Assign unassigned packages
        unassigned_packages = np.argwhere(delivered_packages == False).flatten()
        for p_idx in unassigned_packages:
            p_weight = packages[p_idx][0]
            available_trucks = np.argwhere(initial_capacity >= p_weight).flatten()
            if len(available_trucks) > 0:
                for t_idx in available_trucks:
                    for trip in range(3):
                        assigned_package_weights = [packages[p_idx][0] if p_idx != -1 else 0 for p_idx in current_assignments[t_idx][trip]]
                        if sum(assigned_package_weights) + p_weight <= initial_capacity[t_idx]:
                            for i, delivery in enumerate(current_assignments[t_idx][trip]):
                                if delivery == -1:
                                    current_assignments[t_idx][trip][i] = p_idx
                                    delivered_packages[p_idx] = True

                                    # Recalculate the cost after assigning the package
                                    current_cost, _ = calculate_cost(current_assignments, initial_capacity, packages)
                                    break
                            if delivered_packages[p_idx]:
                                break

    return current_assignments, current_cost, initial_capacity

def print_optimized_assignments(assignments, initial_capacity, packages):
    for t, trips in enumerate(assignments):
        print(f"Truck {t}:")
        i=1
        for trip_idx, trip in enumerate(trips):
            trip_package_weights = [packages[p][0] for p in trip if p != -1]
            if len(trip_package_weights) == 0:
                continue
            truck_capacity = initial_capacity[t]
            print(f"  Trip {i}: weight = {truck_capacity}, package weights = {trip_package_weights}")
            i+=1


# Run the optimization
assignments, remaining_capacity = initial_assignment(trucks_capacity, packages)
initial_cost, _ = calculate_cost(assignments, trucks_capacity, packages)
print("Initial cost:", initial_cost)

optimized_assignments, optimized_cost, initial_capacity = optimization(assignments, trucks_capacity, packages, initial_cost)
print("Optimized cost:", optimized_cost)

# Print truck deliveries
print("\nTruck deliveries after optimization:")

print_optimized_assignments(optimized_assignments, initial_capacity, packages)

# Find and print unassigned packages' weights
_, delivered_packages = calculate_cost(optimized_assignments, initial_capacity, packages)
unassigned_packages = np.argwhere(delivered_packages == False).flatten()
unassigned_weights = [packages[p_idx][0] for p_idx in unassigned_packages]
print("\nUnassigned packages' weights:", unassigned_weights)



Initial cost: 1240.881162696105
Iteration 3: Switched package 3 of truck 3 trip 2 with package 2 of truck 37 trip 1, old cost: 1240.881162696105, new cost: 1239.193662696105
Iteration 6: Switched package 2 of truck 29 trip 2 with package 1 of truck 6 trip 1, old cost: 1239.193662696105, new cost: 1238.5013550037972
Iteration 54: Switched package 3 of truck 31 trip 1 with package 1 of truck 5 trip 3, old cost: 1238.5013550037972, new cost: 1235.8527063551487
Iteration 57: Switched package 1 of truck 5 trip 2 with package 3 of truck 37 trip 3, old cost: 1235.8527063551487, new cost: 1234.1027063551487
Iteration 92: Switched package 1 of truck 8 trip 1 with package 2 of truck 39 trip 3, old cost: 1234.1027063551487, new cost: 1232.1615298845604
Iteration 104: Switched package 1 of truck 9 trip 1 with package 2 of truck 37 trip 2, old cost: 1232.1615298845604, new cost: 1231.5677798845604
Iteration 125: Switched package 2 of truck 17 trip 1 with package 1 of truck 38 trip 3, old cost: 1231